# Packages

In [1]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2

In [41]:
# Setup paths
import rootutils
import xarray as xr
import pandas as pd
root_path = rootutils.find_root()
from burned_embedder.data import load_s2, load_s1_filtered, calculate_search_dates

In [3]:
def img_interactive(da):
    import ipywidgets as widgets
    import matplotlib.pyplot as plt

    def plot_time_steps(time_index1, time_index2):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Left image
        da.isel(time=time_index1).plot.imshow(robust=True, ax=ax1)
        ax1.set_title(f"{str(da.time.values[time_index1])[:10]}")
        ax1.axis('off')
        
        # Right image  
        da.isel(time=time_index2).plot.imshow(robust=True, ax=ax2)
        ax2.set_title(f"{str(da.time.values[time_index2])[:10]}")
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()

    widgets.interact(
        plot_time_steps, 
        time_index1=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=0, description='Left Image'),
        time_index2=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=1, description='Right Image')
    )

In [4]:
deforest_df = pd.read_parquet(root_path / "data" / "processed" / "radd" / "south_america_combined_clean_sampled.parquet")
deforest_df

,Unnamed: 0,geometry,component_id,size_pixels,area_hectares,earliest_alert,latest_alert,duration_days,centroid_x,centroid_y,confidence,tile_name,date_raster_file
0,0,b'\x01\x01\x00\x00\x00F}\x92;l\xbeH\xc0\x9e\x0...,32,109,1.342989,2021-08-29,2021-09-22,24,-49.487678,-8.150868,high,00N_050W_radd_alerts,sample_07_dates.tif
1,1,b'\x01\x01\x00\x00\x00\x91\xd3\xd7\xf35\xcdH\x...,111,312,3.844152,2025-05-16,2025-06-09,24,-49.603209,-8.475450,high,00N_050W_radd_alerts,sample_07_dates.tif
2,2,b'\x01\x01\x00\x00\x00\x80\xee\xcb\x99\xed\xc6...,128,104,1.281384,2025-06-21,2025-07-15,24,-49.554126,-8.481419,high,00N_050W_radd_alerts,sample_07_dates.tif
3,3,b'\x01\x01\x00\x00\x00\xd3jH\xdccYF\xc0we\x17\...,25,1220,15.031620,2020-05-27,2020-06-25,29,-44.698360,-5.696953,high,00N_050W_radd_alerts,sample_11_dates.tif
4,4,b'\x01\x01\x00\x00\x00\x8c\xa2\x07>\x06kF\xc0\...,145,106,1.306026,2022-11-06,2022-11-18,12,-44.836128,-5.717239,high,00N_050W_radd_alerts,sample_11_dates.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,2666,b'\x01\x01\x00\x00\x00\xfa_\xaeE\x0b\x82L\xc0j...,2340,247,3.043287,2021-07-23,2021-08-16,24,-57.015969,-22.889063,high,20S_060W_radd_alerts,sample_36_dates.tif
2666,2667,b'\x01\x01\x00\x00\x00\xdd?\x16\xa2C\xe0O\xc0\...,624,112,1.379952,2020-10-03,2020-10-28,25,-63.752064,-21.886446,high,20S_070W_radd_alerts,sample_01_dates.tif
2667,2668,b'\x01\x01\x00\x00\x00\xa8\x1eip[\x08P\xc0\xab...,12,110,1.355310,2021-10-22,2021-11-16,25,-64.130581,-22.331562,high,20S_070W_radd_alerts,sample_28_dates.tif
2669,2670,b'\x01\x01\x00\x00\x00a\x1c\\:\xe6\xacO\xc0j\x...,172,450,5.544450,2020-09-28,2020-10-22,24,-63.350776,-20.649194,high,20S_070W_radd_alerts,sample_37_dates.tif


In [49]:
index = 2418
p_or_n = "negative"
print(f"Earliest alert: {deforest_df.loc[index, 'earliest_alert']}")
print(f"Latest alert: {deforest_df.loc[index, 'latest_alert']}")

sample = deforest_df.loc[index]

Earliest alert: 2023-08-20
Latest alert: 2023-09-01


In [50]:
search_start_date, search_end_date = calculate_search_dates(
    sample.earliest_alert, sample.latest_alert, 1
)

print(f"Search window: {search_start_date} to {search_end_date}")

Search window: 2023-07-21 to 2023-10-01


In [51]:
load_s1_filtered(lat=sample.centroid_y, lon=sample.centroid_x,
                    deforest_start=sample.earliest_alert,
                    deforest_end=sample.latest_alert,
                    start_date=search_start_date,
                    end_date=search_end_date,
                    n_before=1, n_after=1,
                    edge_size=128)

<xarray.DataArray 'sentinel-1-rtc' (time: 2, band: 2, y: 128, x: 128)> Size: 524kB
dask.array<getitem, shape=(2, 2, 128, 128), dtype=float64, chunksize=(1, 1, 128, 128), chunktype=numpy.ndarray>
Coordinates: (12/40)
  * time                                   (time) datetime64[ns] 16B 2023-07-...
    id                                     (time) <U66 528B 'S1A_IW_GRDH_1SDV...
  * band                                   (band) <U2 16B 'vv' 'vh'
  * x                                      (x) float64 1kB 6.288e+05 ... 6.3e+05
  * y                                      (y) float64 1kB 8.224e+06 ... 8.22...
    constellation                          <U10 40B 'Sentinel-1'
    ...                                     ...
    sar:resolution_azimuth                 int64 8B 22
    description                            (band) <U173 1kB 'Terrain-correcte...
    title                                  (band) <U41 328B 'VV: vertical tra...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32720
    cubo:distance_from_center              (y, x) float64 131kB 901.3 ... 894.8
Attributes:
    collection:           sentinel-1-rtc
    stac:                 https://planetarycomputer.microsoft.com/api/stac/v1
    epsg:                 32720
    resolution:           10
    edge_size:            128
    edge_size_m:          1280
    central_lat:          -16.06234
    central_lon:          -61.790116
    central_y:            8223790.801186335
    central_x:            629415.378816074
    time_coverage_start:  2023-07-21
    time_coverage_end:    2023-10-01

In [ ]:

da_s2 = xr.open_dataarray(root_path / "data" / "processed" / "sen_data" / p_or_n / f"event_{i}" /"da_s2.nc" ) 
da_s2_rgb = da_s2.sel(band=["B04", "B03", "B02"])
img_interactive(da_s2_rgb)

Earliest alert: 2023-11-13
Latest alert: 2023-12-07


FileNotFoundError: [Errno 2] No such file or directory: '/net/home/lmaecker/own_projects/burned_embedder/data/processed/sen_data/negative/event_8/da_s2.nc'